# Starbucks Capstone Challenge - Data Preparation

## Introduction

## Setup

In [1]:
import sys

!{sys.executable} -m pip install -e ../ --quiet

ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: C:\ProgramData\scoop\apps\vscode


In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sb_capstone.wrangling import (
    clean_portfolio,
    clean_profile,
    clean_transcript
)

from sb_capstone.shaping import (
    get_transcript_group,
)

In [3]:
portfolio = pd.read_csv('../data/processed/portfolio.csv')
portfolio = clean_portfolio(portfolio)
portfolio.head()

,id,offer_type,channels,reward,difficulty,duration
0,1,bogo,"[email, mobile, social]",10,10,7
1,2,bogo,"[web, email, mobile, social]",10,10,5
2,3,informational,"[web, email, mobile]",0,0,4
3,4,bogo,"[web, email, mobile]",5,5,7
4,5,discount,"[web, email]",5,20,10


In [4]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   id          10 non-null     int64   
 1   offer_type  10 non-null     category
 2   channels    10 non-null     object  
 3   reward      10 non-null     int64   
 4   difficulty  10 non-null     int64   
 5   duration    10 non-null     int64   
dtypes: category(1), int64(4), object(1)
memory usage: 670.0+ bytes


In [5]:
profile = pd.read_csv('../data/processed/profile.csv')
profile = clean_profile(profile)
profile.head()

,id,gender,age,income,became_member_on
0,1,NaN,NaN,NaN,2017-02-12
1,2,F,55.0,112000.0,2017-07-15
2,3,NaN,NaN,NaN,2018-07-12
3,4,F,75.0,100000.0,2017-05-09
4,5,NaN,NaN,NaN,2017-08-04


In [6]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                17000 non-null  int64         
 1   gender            14825 non-null  category      
 2   age               14825 non-null  float64       
 3   income            14825 non-null  float64       
 4   became_member_on  17000 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), float64(2), int64(1)
memory usage: 548.1 KB


In [7]:
transcript = pd.read_csv('../data/processed/transcript.csv')
transcript = clean_transcript(transcript)
transcript.head()

,person_id,event,time,offer_id,amount,reward
0,4,offer_received,0,4,NaN,NaN
1,4,offer_viewed,6,4,NaN,NaN
2,4,transaction,132,0,19.89,NaN
3,4,offer_completed,132,4,NaN,5.0
4,4,transaction,144,0,17.78,NaN


In [8]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   person_id  306534 non-null  int64   
 1   event      306534 non-null  category
 2   time       306534 non-null  int64   
 3   offer_id   306534 non-null  category
 4   amount     138953 non-null  float64 
 5   reward     33579 non-null   float64 
dtypes: category(2), float64(2), int64(2)
memory usage: 9.9 MB


## Data Transformation

To help us compute the success of each offer, we need to simplify and compress the transcript by summarizing the timeline and summarizing them by extracting their offer group and offer type, because duplicate offers can occur on a single customer's timeline. If the customer has not received any offer, set the offer group and type to zero. 

Additional to that, we need to set if the offer was completed. The challenge for that is that informational offers does not have completion event, but as the presumption in the details of the data, that customers are influenced from the date of its validity. Therefore, we can set the completion event for informational offers if a transaction happens within its validity period.

In [9]:
transcript_group = transcript \
    .merge(portfolio, left_on="offer_id", right_on="id", how="left") \
    .rename(columns={"reward_x": "reward", "reward_y": "offer_reward"})

transcript_group.head()

,person_id,event,time,offer_id,amount,reward,id,offer_type,channels,offer_reward,difficulty,duration
0,4,offer_received,0,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
1,4,offer_viewed,6,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
2,4,transaction,132,0,19.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,offer_completed,132,4,NaN,5.0,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0
4,4,transaction,144,0,17.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
transcript_group = transcript_group.transform(get_transcript_group)
transcript_group[transcript_group.person_id == 4]

,person_id,event,time,offer_id,amount,reward,id,offer_type,channels,offer_reward,difficulty,duration,offer_group
0,4,offer_received,0,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1.0
1,4,offer_viewed,6,4,NaN,NaN,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1.0
2,4,transaction,132,4,19.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,4,offer_completed,132,4,NaN,5.0,4.0,bogo,"[web, email, mobile]",5.0,5.0,7.0,1.0
4,4,transaction,144,0,17.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5,4,offer_received,168,8,NaN,NaN,8.0,informational,"[email, mobile, social]",0.0,0.0,3.0,2.0
6,4,offer_viewed,216,8,NaN,NaN,8.0,informational,"[email, mobile, social]",0.0,0.0,3.0,2.0
7,4,transaction,222,8,19.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
8,4,transaction,240,0,29.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9,4,transaction,378,0,23.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [24]:
transcript_group \
    .sort_values(by=["person_id", "time", "event"]) \
    .groupby(["person_id", "offer_group"]) \
    .agg({
        "event": lambda x: x.tolist(), 
        "offer_id": "min", 
        "amount": "sum", 
        "reward": "min", 
        "offer_type": "first", 
        "channels": "first",
        "offer_reward": "min",
        "difficulty": "min",
        "duration": "min"
        }) \
    .reset_index()

,person_id,offer_group,event,offer_id,amount,reward,offer_type,channels,offer_reward,difficulty,duration
0,1,1.0,"[offer_received, offer_viewed]",10,0.00,NaN,discount,"[web, email, mobile]",2.0,10.0,7.0
1,1,2.0,"[offer_received, offer_viewed, transaction]",5,0.35,NaN,discount,"[web, email]",5.0,20.0,10.0
2,1,3.0,"[offer_received, offer_viewed, transaction, tr...",7,2.63,2.0,discount,"[web, email, mobile, social]",2.0,10.0,10.0
3,1,4.0,"[offer_received, offer_viewed, transaction, tr...",6,7.54,3.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0
4,1,5.0,"[offer_received, offer_viewed, transaction, tr...",7,9.88,NaN,discount,"[web, email, mobile, social]",2.0,10.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
90490,17000,0.0,"[transaction, transaction, transaction, transa...",0,90.90,NaN,NaN,None,NaN,NaN,NaN
90491,17000,1.0,"[offer_received, offer_viewed, offer_completed...",6,21.55,3.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0
90492,17000,2.0,[offer_received],3,0.00,NaN,informational,"[web, email, mobile]",0.0,0.0,4.0
90493,17000,3.0,"[offer_received, offer_completed, transaction,...",9,30.57,5.0,bogo,"[web, email, mobile, social]",5.0,5.0,5.0


In [12]:
transcript_group[transcript_group.person_id==1]

,person_id,event,time,offer_id,amount,reward,id,offer_type,channels,offer_reward,difficulty,duration,offer_group
277981,1,offer_received,168,10,NaN,NaN,10.0,discount,"[web, email, mobile]",2.0,10.0,7.0,1.0
277982,1,offer_viewed,216,10,NaN,NaN,10.0,discount,"[web, email, mobile]",2.0,10.0,7.0,1.0
277983,1,offer_received,336,5,NaN,NaN,5.0,discount,"[web, email]",5.0,20.0,10.0,2.0
277984,1,offer_viewed,348,5,NaN,NaN,5.0,discount,"[web, email]",5.0,20.0,10.0,2.0
277985,1,transaction,360,5,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
277986,1,offer_received,408,7,NaN,NaN,7.0,discount,"[web, email, mobile, social]",2.0,10.0,10.0,3.0
277987,1,offer_viewed,408,7,NaN,NaN,7.0,discount,"[web, email, mobile, social]",2.0,10.0,10.0,3.0
277988,1,transaction,414,7,0.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
277989,1,transaction,444,7,1.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
277990,1,offer_received,504,6,NaN,NaN,6.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0,4.0


In [13]:
transcript_group[transcript_group.person_id==17000]

,person_id,event,time,offer_id,amount,reward,id,offer_type,channels,offer_reward,difficulty,duration,offer_group
238065,17000,offer_received,0,6,NaN,NaN,6.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0,1.0
238066,17000,offer_viewed,6,6,NaN,NaN,6.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0,1.0
238067,17000,transaction,54,6,21.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
238068,17000,offer_completed,54,6,NaN,3.0,6.0,discount,"[web, email, mobile, social]",3.0,7.0,7.0,1.0
238069,17000,transaction,84,0,25.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
238070,17000,transaction,96,0,21.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
238071,17000,offer_received,336,3,NaN,NaN,3.0,informational,"[web, email, mobile]",0.0,0.0,4.0,2.0
238072,17000,offer_received,408,9,NaN,NaN,9.0,bogo,"[web, email, mobile, social]",5.0,5.0,5.0,3.0
238073,17000,transaction,480,9,30.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
238074,17000,offer_completed,480,9,NaN,5.0,9.0,bogo,"[web, email, mobile, social]",5.0,5.0,5.0,3.0


In [14]:
transcript[transcript.person_id==17000]

,person_id,event,time,offer_id,amount,reward
238065,17000,offer_received,0,6,NaN,NaN
238066,17000,offer_viewed,6,6,NaN,NaN
238067,17000,transaction,54,0,21.55,NaN
238068,17000,offer_completed,54,6,NaN,3.0
238069,17000,transaction,84,0,25.19,NaN
238070,17000,transaction,96,0,21.53,NaN
238071,17000,offer_received,336,3,NaN,NaN
238072,17000,offer_received,408,9,NaN,NaN
238073,17000,transaction,480,0,30.57,NaN
238074,17000,offer_completed,480,9,NaN,5.0
